In [31]:
import pandas as pd
import numpy as np

import re

from utils import tag_mail, format_mail

In [32]:
df = pd.read_json(r'data/gmail_azoulai_basic.txt', encoding = 'utf-8')

In [33]:
df['cat'] = df.headers.apply(tag_mail)

df.loc[df.cat=='doctolib']

,snippet,date,headers,body,cat
155,Doctolib RDV confirmé Karine Flamand Psycholog...,2020-01-09 13:39:15,"[{'name': 'From', 'value': 'Doctolib <no-reply...",,doctolib
388,Doctolib Rappel RDV Dr William ECLANCHER Médec...,2019-10-15 11:00:16,"[{'name': 'From', 'value': 'Doctolib <no-reply...",,doctolib
399,Doctolib RDV confirmé Dr William ECLANCHER Méd...,2019-10-11 14:55:22,"[{'name': 'From', 'value': 'Doctolib <no-reply...",,doctolib
418,Doctolib RDV confirmé Dr William ECLANCHER Méd...,2019-10-08 16:03:03,"[{'name': 'From', 'value': 'Doctolib <no-reply...",,doctolib


In [48]:
mails_doc = df.loc[df.cat == 'doctolib']
mails_doc = mails_doc.loc[mails_doc.snippet.str.contains("confirmé")]

mails_doc['docteur'] = mails_doc.snippet.apply(lambda x: re.compile(r' Lundi|Mardi|Mercredi|Jeudi|Vendredi|Samedi|Dimanche').split(x.split('confirmé ')[1])[0] )

for el in mails_doc.snippet:
    print(el)
    tmp = el
    tmp = tmp.split('confirmé ')[1]
    tmp = re.compile(r' Lundi|Mardi|Mercredi|Jeudi|Vendredi|Samedi|Dimanche').split(tmp)[0]
    name_doc = tmp
    
    print(tmp)
    print('\n\n')
    print('#####################################################')
    print('\n\n')

mails_doc

Doctolib RDV confirmé Karine Flamand Psychologue Jeudi 09 janvier à 18h45 DÉPLACER RDV ANNULER RDV En confirmant ce rendez-vous, vous vous engagez à vous y rendre. Pensez bien à annuler le plus tôt
Karine Flamand Psychologue 



#####################################################



Doctolib RDV confirmé Dr William ECLANCHER Médecin généraliste Mercredi 16 octobre à 09h15 DÉPLACER RDV ANNULER RDV Moyens de paiement et remboursement Chèques Espèces Cartes Bancaires Merci de vous
Dr William ECLANCHER Médecin généraliste 



#####################################################



Doctolib RDV confirmé Dr William ECLANCHER Médecin généraliste Mercredi 09 octobre à 11h15 DÉPLACER RDV ANNULER RDV Moyens de paiement et remboursement Chèques Espèces Cartes Bancaires Merci de vous
Dr William ECLANCHER Médecin généraliste 



#####################################################





,snippet,date,headers,body,cat,docteur
155,Doctolib RDV confirmé Karine Flamand Psycholog...,2020-01-09 13:39:15,"[{'name': 'From', 'value': 'Doctolib <no-reply...",,doctolib,Karine Flamand Psychologue
399,Doctolib RDV confirmé Dr William ECLANCHER Méd...,2019-10-11 14:55:22,"[{'name': 'From', 'value': 'Doctolib <no-reply...",,doctolib,Dr William ECLANCHER Médecin généraliste
418,Doctolib RDV confirmé Dr William ECLANCHER Méd...,2019-10-08 16:03:03,"[{'name': 'From', 'value': 'Doctolib <no-reply...",,doctolib,Dr William ECLANCHER Médecin généraliste
